In [3]:
import pandas as pd

df = pd.read_csv('nodes')
df=df['specialty;"city";"patients";"free_time";"community";"friends";"adoption_date";"proximity";"med_sch_yr";"jours";"clubs";"meetings";"id";"discuss"'].str.split(';', expand=True)


In [4]:
'''
Global configurations
'''
k_alpha = 1     #frienship relation
k_beta = .5     #advice relation
k_delta = .6    #discussion relation
E_profit = 10
K_self_coupling = -2

num_clusters = 5
num_agents = 20

# **rearrange connections into clusters**

In [5]:
num_agents = df.shape[0]
print(num_agents)

246


In [6]:
import csv

import numpy as np


cluster_max_index = [116, 166, 210, 246]
cluster_range = [[0,116],[116,166],[166,210],[210,246]]

cluster_names = ["Peoria","Bloomington" , "Quincy" ,"Galesburg"]



def get_edges_array(cluster_max):

    peoria_cluster_edges = []

    bloomington_cluster_edges = []

    quincy_cluster_edges = []

    galesburg_cluster_edges = []

    with open('medical_innovationver3.csv',newline='', encoding='utf-8') as f:

        reader = csv.reader(f)

        row_num = 0

        for row in reader:
            if row_num == 0:
                row_num+=1
            else:
                edge_properties = []
                args = row[0].split(";")

                node1 = int(args[0])-1

                

                node2 = int(args[1][1:-1])-1

                fr_ad_dis = [int(args[2][1:-1]), int(args[3][1:-1]), int(args[5][1:-1])]

                id = int(args[4][1:-1])

                edge_properties= [node1, node2, fr_ad_dis[0], fr_ad_dis[1], fr_ad_dis[2]]

                if (node1 <= cluster_max[0] and node2 <= cluster_max[0]):

                    

                    peoria_cluster_edges.append(edge_properties)

                elif(node1 <= cluster_max[1] and node2 <= cluster_max[1]): 

                    bloomington_cluster_edges.append(edge_properties)

                elif(node1 <= cluster_max[2] and node2 <= cluster_max[2]): 

                    quincy_cluster_edges.append(edge_properties)

                else: 

                    galesburg_cluster_edges.append(edge_properties)

    return peoria_cluster_edges, bloomington_cluster_edges, quincy_cluster_edges, galesburg_cluster_edges    



###

# edges arrays arguments: 

#   - node1 (from)

#   - node2 (to) 

#   - friendship relation 

#   - advice relation

#   - discussion relation

#  ###

peoria_edges, bloomington_edges, quincy_edges, galesburg_edges = get_edges_array(cluster_max_index)

# **get time revolution (of opinion status of an agent) with original data**

1.   each row represents an agent
2.   each column represents the agent's opinion at this timepoint



In [7]:
ad_time_all = df[6]
for i in range(246):
  ad_time_all[i] = int(ad_time_all[i][1:-1])
print(ad_time_all)

0      99
1      99
2      99
3      98
4      98
       ..
241     9
242     9
243     8
244    12
245     1
Name: 6, Length: 246, dtype: object


In [8]:


time_range = ad_time_all.max()
print(time_range)

opinion_timeline_agents = []
for i in range(num_agents):
  opinion_timeline_agent_i = np.zeros(99)
  adoption_timepoint = int(ad_time_all[i])
  opinion_timeline_agent_i=[1 if j >= adoption_timepoint-1 else 0 for j in range(time_range) ] 
  opinion_timeline_agents.append(opinion_timeline_agent_i)
opinion_timeline_agents=np.asarray(opinion_timeline_agents)

print(opinion_timeline_agents)

99
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


# **initial opinion state**

In [9]:
initial_opinions = []

for i in range(num_agents): 
    initial_opinions.append(opinion_timeline_agents[i][0])

print(initial_opinions)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


# **initial opinions per cluster**

In [16]:
max_node_array = [116, 166, 210, 246]
cluster_range = [[0,116],[116,166],[166,210],[210,246]]

cluster_names = ["Peoria","Bloomington" , "Quincy" ,"Galesburg"]


peoria_initials = initial_opinions[0:max_node_array[0]]
bloomington_initials = initial_opinions[cluster_range[1][0]:cluster_range[1][1]]
quincy_initials = initial_opinions[cluster_range[2][0]:cluster_range[2][1]]
galesburg_initials = initial_opinions[cluster_range[3][0]:cluster_range[3][1]]
opinion_initials_per_cluster = [peoria_initials, bloomington_initials, quincy_initials, galesburg_initials]

def get_initial_opinions(cluster_number): 
    return opinion_initials_per_cluster[cluster_number]



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


# driving force custom

In [34]:

def update_driving_forces_cluster_speciific(edge_props, cluster_number, opinions): 
    len_cluster = 0
    first_node = 0
    if cluster_number == 0: 
        len_cluster = max_node_array[0]
    else: 
        len_cluster = max_node_array[cluster_number]-max_node_array[cluster_number-1]
        first_node = max_node_array[cluster_number-1]
    
    updated_driving_force = np.zeros(len_cluster)

    agents_ids = [first_node+i for i in range(len_cluster)]

    for a in agents_ids:
        for edge in edge_props: 
           #check that the edge is directed at node a and opinions of agents don't match
           if edge[1] == a and  not opinions[a-first_node] == opinions[edge[0]-first_node]: 
                updated_driving_force[a-first_node] += edge[2] * k_alpha + edge[3] * k_beta + edge[4]*k_delta
        updated_driving_force[a-first_node]+=E_profit
        print("agent {} obtained a driving force of {}".format(a, updated_driving_force[a]))

    return updated_driving_force 

d_forces = update_driving_forces_cluster_speciific(peoria_edges, 0, cluster_max_index , peoria_initials )
print(d_forces)

agent 0 obtained a driving force of 10.0
agent 1 obtained a driving force of 10.0
agent 2 obtained a driving force of 10.0
agent 3 obtained a driving force of 10.0
agent 4 obtained a driving force of 10.0
agent 5 obtained a driving force of 10.0
agent 6 obtained a driving force of 10.0
agent 7 obtained a driving force of 10.0
agent 8 obtained a driving force of 10.0
agent 9 obtained a driving force of 10.0
agent 10 obtained a driving force of 10.0
agent 11 obtained a driving force of 10.0
agent 12 obtained a driving force of 10.0
agent 13 obtained a driving force of 10.0
agent 14 obtained a driving force of 10.0
agent 15 obtained a driving force of 10.0
agent 16 obtained a driving force of 10.0
agent 17 obtained a driving force of 10.0
agent 18 obtained a driving force of 10.5
agent 19 obtained a driving force of 10.0
agent 20 obtained a driving force of 10.0
agent 21 obtained a driving force of 10.0
agent 22 obtained a driving force of 10.0
agent 23 obtained a driving force of 10.0
ag

# main function 

In [ ]:
'''
[cluster_elements, driving_forces, agents, opinions]
'''
def main():
    elems = create_clusters(num_clusters, num_agents, cluster_specific=True)
    probs = change_probs(elems[1])
    
    cluster_elements, driving_forces, agents, opinions = elems[0], elems[1], elems[2], elems[3]   
    # [cluster_elements, driving_forces, agents, opinions]
    
    # cs: [cluster_elements, driving_forces, agents, opinions]
    #  current_state: (agents' instrinsics, opinions, probabilities of change, driving forces)
    current_state = [agents, opinions, probs, driving_forces]
    for x in range(5):
        print("iteration", x)
        #visualise([init[0], current_state[3], current_state[0], current_state[1]])
        print(driving_forces)
        current_state[3] = update_driving_forces(clusters, driving_forces, agents, opinions, update_agents=False)
        current_state = update_state(current_state)

        

main()

# probability calculation

In [ ]:
'''
compute the probability of change given a driving force    
'''
def change_probs(driving_forces):
    hi = np.sum(driving_forces) # the maximum possible driving force 
    lo = None
    driving_forces = normalise_driving_force(driving_forces, hi )
    probs = [math.erf(driving_forces[i]) if driving_forces[i] < hi else math.erf(hi) for i in range(len(driving_forces)) ]
    return probs


'''
The new opinion {x} of an agent given the prob. of change {p}

Can serve as a state transition function
'''
def opinion(x, p):
    res = (np.random.rand(1))[0]
    return 1-x if res < p else x

'''
Part of modelling the transition function

'''
def normalise_driving_force(driving_forces, hi):
    return driving_forces/hi


'''
You can define state transition functions here
'''
def transition_function(driving_forces, opinions, cluster_number, edge_props):
       agents = state[0]
        #print(state)
       state[2] = change_probs(state[3])

       for i in range(len(agents)): 
              new_opinion = opinion(state[1][i], state[2][i])
              state[1][i] = new_opinion
       return state

